### PART 1: Import datacube

In [ ]:
# importing module
import os
import sys
import numpy as np
import py4DSTEM
from py4DSTEM import show
import matplotlib.pyplot as plt
import hyperspy.api as hs

In [ ]:
#%matplotlib widget 

In [ ]:
s = hs.load(filepaths[str(filepath_index)], reader='HSPY')
datacube = py4DSTEM.DataCube(m.data) 

### PART 2: Add noise

In [ ]:
%run Noise_noise_models.ipynb

In [ ]:
# Example 1: Single noise source
noisy_dc = add_noise_to_datacube(
    datacube,
    [(PoissonNoise(), {'scale': 50})]
)

# Example 2: Multiple noise sources
noisy_dc = add_noise_to_datacube(
    datacube,
    [
        (PoissonNoise(), {'scale': 100}),
        (GaussianNoise(), {'sigma': 2}),
        (ReadoutNoise(), {'sigma': 5})
    ],
    seed=42
)

# Example 3: Realistic detector noise
noisy_dc = add_realistic_detector_noise(
    datacube,
    dose_scale=80,
    readout_sigma=3.5,
    dark_current=0.5
)

# Example 4: Custom noise (Multiple sources + low_dose)
noisy_dc = add_noise_to_datacube(
    reduce_dose(datacube, dose_fraction=0.3, method='electron_counting'),
    [
        (PoissonNoise(), {'scale': 80}),
        (DarkCurrentNoise(), {'dark_current': 0.5}),
        (ReadoutNoise(), {'sigma': 3.5}),
    ]
)

# Example 5: Bimodal sparse data (80% zeros + Gaussian signal at 30)
low_dose_bimodal = reduce_dose(
    datacube,
    dose_fraction=0.2,
    method='bimodal',
    signal_mean=30.0,
    signal_sigma=10.0
)

In [ ]:
# Visualize effect of noise in Q space
plt.figure(figsize=(5,5))
plt.imshow(noisy_dc[0,0,:,:], vmax = 10, cmap='gray')

In [ ]:
# Visualizedifference between original and noisy datacube in Q space
plt.figure(figsize=(5,5))
plt.imshow(noisy_dc[0,0,:,:] - datacube[0,0,:,:], cmap='gray')

### PART 3: Compare original and noisy

In [ ]:
noisy_present = True

In [ ]:
#Calculate mean and max diffraction patterns in Q
dp_mean = datacube.get_dp_mean()
dp_max = datacube.get_dp_max()
if noisy_present == True:
    dp_mean_noisy = noisy_dc.get_dp_mean()
    dp_max_noisy = noisy_dc.get_dp_max()

In [ ]:
# Find the center and probe radius
# Get the probe position and size. Note that if the datacube is overexposed we need to use a reference datacube
probe_semiangle, probe_qx0, probe_qy0 = datacube.get_probe_size(dp_mean.data,thresh_lower=0.0001,thresh_upper=0.05,N=50)

# Overlay the computed probe over the maximum diffraction pattern
show(dp_max,scaling='log',circle={'center':(probe_qx0, probe_qy0),'R':probe_semiangle,'alpha':0.3,'fill':True})

# Print the estimated probe radius
print('Estimated probe radius =', '%.2f' % probe_semiangle, 'pixels')

In [ ]:
# Position the virtual annular detector. Radii can be changed to visualize different features
center = probe_qx0, probe_qy0
r_inner = probe_semiangle * 6.8
r_outer = probe_semiangle * 8.8
radii = r_inner,r_outer

# overlay selected detector position over mean dp
datacube.position_detector(mode='annular',geometry=(center,radii))

In [ ]:
# Capture the virtual ADF and BF to select reference and analysis ROIs
datacube.get_virtual_image(mode='annulus',geometry=(center,radii),name='annular_dark_field')
datacube.get_virtual_image(mode='circle',geometry=(center,probe_semiangle),name='bright_field',)
if noisy_present == True:
    noisy_dc.get_virtual_image(mode='annulus',geometry=(center,radii),name='annular_dark_field')
    noisy_dc.get_virtual_image(mode='circle',geometry=(center,probe_semiangle),name='bright_field',)


In [ ]:
print("Original: ")
show(datacube.tree('annular_dark_field'))
if noisy_present == True:
    print("Noisy: ")
    show(noisy_dc.tree('annular_dark_field'))
#plt.imsave(r"PATH.tiff",datacube.tree('annular_dark_field').data, cmap='gray')

In [ ]:
print("Original: ")
show(datacube.tree('bright_field'))
if noisy_present == True:
    print("Noisy: ")
    show(noisy_dc.tree('bright_field'))

In [ ]:
if noisy_present == True:
    bf_difference = noisy_dc.tree('bright_field').data - datacube.tree('bright_field').data
    adf_difference = noisy_dc.tree('annular_dark_field').data - datacube.tree('annular_dark_field').data
    plt.figure(figsize=(6,6))
    plt.imshow(adf_difference)

### Save noisy datacube

In [ ]:
# Force uint8 and matching chunks
save_data = hs.signals.Signal2D(low_dose_bimodal.data.astype(np.uint8))
save_data.save(r'YOUR_PATH\synth_noise_datacube.hdf5', file_format='HSPY', overwrite=True, compression='gzip', compression_opts=4, chunks=(2, 4, 257, 257))